In [ ]:
import numpy as np 
import pandas as pd 

import tensorflow as tf 
import re
from keras import backend as K 
from tensorflow.keras.preprocessing.sequence import pad_sequences 
import tensorflow.keras.layers as L 
from tensorflow.keras.optimizers import Adam 

from tensorflow.keras.losses import SparseCategoricalCrossentropy 
from sklearn.model_selection import train_test_split 
from tensorflow.keras.preprocessing import image 
from PIL import Image 

from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D 
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization 
from google.colab import drive


In [ ]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/Pokemon/Pokemon/

/content/drive/My Drive/Pokemon/Pokemon


In [ ]:
pokemons = pd.read_csv('pokemon-images-and-types/pokemon.csv')
numbers = []
for i in range(1,pokemons.shape[0]+1):
    numbers.append(i)
pokemons['pkn'] = numbers
IMG_DIR = 'pokemon-images-dataset/pokemon'
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(IMG_DIR) if isfile(join(IMG_DIR, f))]

In [ ]:
# Since we will need type2, we will fill it with type1 if it is na
def dualTypeMaker(type1, type2):
    if pd.isnull(type2):
        return type1 
    return type2 

pokemons["Type2"] = pokemons.apply(lambda x: dualTypeMaker(x["Type1"], x["Type2"]), axis = 1) 
pokemons.head()

,Name,Type1,Type2,pkn
0,bulbasaur,Grass,Poison,1
1,ivysaur,Grass,Poison,2
2,venusaur,Grass,Poison,3
3,charmander,Fire,Fire,4
4,charmeleon,Fire,Fire,5


In [ ]:
# Add generations to the dataframe
gen = [] 
length = len(pokemon) 

for i in range(0, length):
    if i<=150:
        gen.append(1)
    elif i<=250:
        gen.append(2)
    elif i<=385:
        gen.append(3)
    elif i<=492:
        gen.append(4)
    elif i<=648:
        gen.append(5)
    elif i<=720:
        gen.append(6)
    else:
        gen.append(7)
        

boarderPokemon = ["mew", "chikorita", "celebi", "treecko", "deoxys-normal", "turtwig", "arceus", "victini", "genesect", "chespin",
                 "volcanion", "rowlet", "melmetal"]
        
pokemons["Gen"] = gen 

pokemons["Gen"] = pokemons["Gen"].astype(str) 

print("Name, Generation\n") 


for i in range(0, length):
    if pokemons["Name"][i] in boarderPokemon: 
        print(pokemons["Name"][i], ", ", pokemons["Gen"][i]) 

pokemons.head() 

Name, Generation

mew ,  1
chikorita ,  2
celebi ,  2
treecko ,  3
deoxys-normal ,  3
turtwig ,  4
arceus ,  4
victini ,  5
genesect ,  5
chespin ,  6
volcanion ,  6
rowlet ,  7
melmetal ,  7


,Name,Type1,Type2,pkn,Gen
0,bulbasaur,Grass,Poison,1,1
1,ivysaur,Grass,Poison,2,1
2,venusaur,Grass,Poison,3,1
3,charmander,Fire,Fire,4,1
4,charmeleon,Fire,Fire,5,1


In [ ]:
numbers = []
for i in range(1,pokemons.shape[0]+1):
    numbers.append(i)
pokemons['pkn'] = numbers
IMG_DIR = 'pokemon-images-dataset/pokemon'
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(IMG_DIR) if isfile(join(IMG_DIR, f))]

In [ ]:
import re
dataframe_img = pd.DataFrame([])
images = []
pokemon_number = []
for img in onlyfiles:
    if not re.search('-', img):
        pkn = img.split('.')
        n = re.sub("[^0-9]", "", pkn[0])
        path = IMG_DIR +'/' +str(img)
        images.append(path)
        pokemon_number.append(n)
dataframe_img['images'] = images
dataframe_img['pkn'] = pokemon_number
dataframe_img['pkn'] = dataframe_img['pkn'].astype(int)
dataframe_img['pkn'] = dataframe_img['pkn'].astype(int)
result = pokemons.merge(dataframe_img, left_on='pkn', right_on='pkn')
result.head()

,Name,Type1,Type2,Gen,pkn,images
0,bulbasaur,Grass,Poison,1,1,pokemon-images-dataset/pokemon/1.png
1,ivysaur,Grass,Poison,1,2,pokemon-images-dataset/pokemon/2.png
2,venusaur,Grass,Poison,1,3,pokemon-images-dataset/pokemon/3.png
3,charmander,Fire,Fire,1,4,pokemon-images-dataset/pokemon/4.png
4,charmeleon,Fire,Fire,1,5,pokemon-images-dataset/pokemon/5.png


In [ ]:
imageTrain, imageTest, genTrain, genTest = train_test_split(result["images"], result["Gen"], test_size = 0.3, random_state = 919)

imageTrain, imageVal, genTrain, genVal = train_test_split(imageTrain, genTrain, test_size = 0.4, random_state = 919)

In [ ]:
datagen = image.ImageDataGenerator()
trainDf = pd.DataFrame(imageTrain) 
trainDf["Gen"] = genTrain 
train = datagen.flow_from_dataframe(trainDf, x_col = "images", y_col = "Gen") 

Found 297 validated image filenames belonging to 6 classes.


In [ ]:
testDf = pd.DataFrame(imageTest) 
testDf["Gen"] = genTest 
test = datagen.flow_from_dataframe(testDf, x_col = "images", y_col = "Gen") 

Found 213 validated image filenames belonging to 6 classes.


In [ ]:
valDf = pd.DataFrame(imageVal) 
valDf["Gen"] = genVal 
val = datagen.flow_from_dataframe(valDf, x_col = "images", y_col = "Gen") 

Found 198 validated image filenames belonging to 6 classes.


In [ ]:



batchSize = 8 #The number of items in each batch
width = 256 #The width of the images
height = 256 #The height of the images
channels = 3 #The number of channels (RGB)

model = Sequential() 
model.add(L.Lambda(lambda x: x, input_shape = (width, height, channels))) #Put the input into a lambda, because it would not work for some reason in the Conv2D
model.add(Conv2D(32, (3, 3))) 
model.add(BatchNormalization()) 
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) 

model.add(Conv2D(32, (3, 3))) 
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 

model.add(Conv2D(64, (3, 3))) 
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Flatten()) 
model.add(Dense(64)) 
model.add(Activation('relu')) 
model.add(Dropout(0.2)) 
model.add(Dense(1)) 
model.add(Activation('sigmoid')) 

model.compile(loss = 'binary_crossentropy', 
              optimizer='adam', metrics=['acc'])

history = model.fit_generator(train, epochs = 5, validation_data=val) 

Epoch 1/5
10/10 [==============================] - 47s 5s/step - loss: 3.3865 - acc: 0.7637 - val_loss: 0.6397 - val_acc: 0.8333
Epoch 2/5
10/10 [==============================] - 30s 3s/step - loss: 0.5936 - acc: 0.8333 - val_loss: 1.2088 - val_acc: 0.8333
Epoch 3/5
10/10 [==============================] - 31s 3s/step - loss: 0.5013 - acc: 0.8333 - val_loss: 0.5120 - val_acc: 0.8333
Epoch 4/5
10/10 [==============================] - 30s 3s/step - loss: 0.4842 - acc: 0.8333 - val_loss: 0.6513 - val_acc: 0.8333
Epoch 5/5
10/10 [==============================] - 30s 3s/step - loss: 0.4759 - acc: 0.8333 - val_loss: 0.5033 - val_acc: 0.8333


In [ ]:
predict = model.predict_generator(test)

Instructions for updating:
Please use Model.predict, which supports generators.


In [ ]:
import plotly.io as pio
pio.renderers.default = 'colab'
import plotly.graph_objects as go
fig = go.Figure()
epochs = []
for i in range(len(history.history['acc'])):
    epochs.append(i)
fig.add_trace(go.Scatter(x=epochs,y=history.history['acc'], mode='lines',name='Training Accuracy'))
fig.add_trace(go.Scatter(x=epochs,y=history.history['val_acc'], mode='lines',name='Validation Accuracy'))
fig.update_layout({
        'plot_bgcolor': 'rgba(0, 0, 0, 0)',
        'paper_bgcolor': 'rgba(0, 0, 0, 0)',
        'width': 500, 
        'height': 400

})
fig.show()

In [ ]:
loss, accuracy = model.evaluate(test)
print(accuracy)

7/7 [==============================] - 5s 679ms/step - loss: 0.4949 - acc: 0.8333
0.8333333134651184
